# Torch GLRT Confidence Intervals

Use GLRT and attribution regularization to calculate CIs for attributions of a Torch model.

Demonstrated here with an OLS model.

The GLRT results should not match bootstrap and analytic for two reasons:
* I'm not sure the GLRT torch code is perfect yet
* Attributions are generally strictly positive, unlike regression coefficients

In [1]:
import numpy as np
import torch
from bootstrapCoefficients import bootstrapCis
from analyticLinearRegressionCIs import analyticLinearCis
from glrtTorch import glrtTorchCis
from DataGeneration import default_data
from sklearn.linear_model import LinearRegression
from torch_linear import TorchLinear

In [2]:
X, y = default_data()

## Sklearn model
Bootstrap and analytic results

In [4]:
LR = LinearRegression()
LR.fit(X, y)
print("Coefficients:", LR.coef_)

print("Bootstrapping")
lcb_LR, ucb_LR = bootstrapCis(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Lower bounds:", lcb_LR, "\nUpper bounds:", ucb_LR)

print("Analytic solution")
lcb_LR_a, ucb_LR_a = analyticLinearCis(LR, X, y, alpha=0.05)
print("Lower bounds:", lcb_LR_a, "\nUpper bounds:", ucb_LR_a)

Coefficients: [0.99906024 1.02556399]
Bootstrapping
Lower bounds: [0.85987423 0.86049457] 
Upper bounds: [1.13705651 1.17785711]
Analytic solution
Lower bounds: [0.85619486 0.86764879] 
Upper bounds: [1.14192561 1.18347918]


# Torch model
Bootstrap and analytic results

In [4]:
TL = TorchLinear(lr=0.003)
TL.fit(X,y)
# Takes ~4min
print("Bootstrapping")
lcb_TL, ucb_TL = bootstrapCis(lambda:TorchLinear(lr=0.003), X=X, y=y, alpha=0.05, replicates=1000)
print("Lower bounds:", lcb_TL, "\nUpper bounds:", ucb_TL)

print("Analytic solution")
lcb_TL_a, ucb_TL_a = analyticLinearCis(TL, X, y, alpha=0.05)
print("Lower bounds:", lcb_TL_a, "\nUpper bounds:", ucb_TL_a)

Bootstrapping
Lower bounds: [0.80575109 0.88677126] 
Upper bounds: [1.06291306 1.20051801]
Analytic solution
Lower bounds: [0.79458542 0.87648867] 
Upper bounds: [1.08140473 1.18232514]


GLRT Results

In [3]:
TL = TorchLinear(lr=0.003)
TL.fit(X,y)
print("GLRT")
lcb_GLRT, ucb_GLRT, lcb_Reults, ucb_Results = glrtTorchCis(lambda:TorchLinear(lr=0.003), X=X, y=y, alpha=0.05,search_kwargs={'lmbds':np.logspace(-10,10,101)})
print("Lower bounds:", lcb_GLRT, "\nUpper bounds:", ucb_GLRT)

GLRT


TypeError: only integer scalar arrays can be converted to a scalar index

In [4]:
%debug

> /home/gabe/projects/explanation-uncertainty/glrtTorch.py(52)getBoundary()
     50 
     51     mses, coefs = np.array(mses), np.array(attributions)
---> 52     return reduction(attributions[mses<=ucb]), (mses,attribs,coefs,biases)
     53 
     54 def trainWithAttributions(model,X,y,obj,lossfunc,lr=0.001,max_iter=1000):



ipdb>  attributions.shape


*** AttributeError: 'list' object has no attribute 'shape'


ipdb>  len(attributions)


101


ipdb>  len(mses)


101


ipdb>  attributions


[0.5756043195724487, 0.6082776784896851, 0.6958274841308594, 0.7042483687400818, 0.6791307330131531, 0.8404797315597534, 0.6548677682876587, 0.7709811329841614, 0.6170388460159302, 0.6623507142066956, 0.5350603461265564, 0.7034501433372498, 0.5963259339332581, 0.6757538318634033, 0.5677726864814758, 0.7518547773361206, 0.8773732781410217, 0.7324469685554504, 0.7724628448486328, 0.7407637238502502, 0.6661356091499329, 0.5521718859672546, 0.847093939781189, 0.6504746079444885, 0.6185213923454285, 0.6759927272796631, 0.680371880531311, 0.8265122771263123, 0.5728380680084229, 0.5609650611877441, 0.7258374094963074, 0.8019088506698608, 0.9052780270576477, 0.7468176484107971, 0.831562340259552, 0.7087811827659607, 0.8030027747154236, 0.8480169773101807, 0.8175663948059082, 0.5433512330055237, 0.7682833671569824, 0.8306447863578796, 0.7278227210044861, 0.8428712487220764, 0.5950493812561035, 0.7110801935195923, 0.7331608533859253, 0.8024799227714539, 0.6348637938499451, 0.912880003452301, 0.9

ipdb>  c


In [8]:
ucb_Results[0][1][:10]

tensor([[-1.7616e+09,  1.6505e+08],
        [-3.4697e+09,  9.9392e+08],
        [ 1.9709e+09, -3.4935e+08],
        [-4.4904e+09,  1.6282e+09],
        [-7.6834e+08,  1.3738e+09],
        [-9.9375e+08,  8.0591e+08],
        [ 3.6013e+09, -5.0505e+07],
        [-1.3904e+09,  7.4591e+08],
        [ 1.9443e+09,  1.3400e+09],
        [-6.6773e+09,  1.4681e+09]], grad_fn=<SliceBackward>)

In [9]:
ucb_Results[0][2][:10]

[array([1.0711303, 0.8208021], dtype=float32),
 array([1.0481999 , 0.89408934], dtype=float32),
 array([1.0149324 , 0.90760565], dtype=float32),
 array([0.97981447, 0.889424  ], dtype=float32),
 array([0.9869582, 0.9537857], dtype=float32),
 array([1.1045136, 0.8575846], dtype=float32),
 array([0.93058103, 1.0066142 ], dtype=float32),
 array([0.8684172, 1.0420791], dtype=float32),
 array([0.8235353, 1.1172069], dtype=float32),
 array([1.0131657, 0.9682913], dtype=float32)]